In [1]:
#settings for notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#show python version
import platform
platform.python_version()



'3.8.5'

In [2]:
import scrapy;
import bs4;
import scrapydo
scrapydo.setup()

In [3]:
import scrapy
from scrapy.crawler import CrawlerProcess

Setup a pipeline

In [20]:
import json
import requests

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('fileoutput.txt', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider): 
        line = json.dumps(dict(item)) + "\n"
        image_url = item['download'][0]
        print(f'Downloading: {image_url}')
        image_req = requests.get(image_url)
        filename = os.path.basename(image_url)
        print(f'saving to: {filename}')
        image = open(f'./backgrounds/{filename}', "wb")
        image.write(image_req.content)
        image.close()
        self.file.write(line)
        return item

In [11]:
import logging

class wallpaperSpider(scrapy.Spider):

    name = 'desktopography-2020'
    start_urls = ['https://desktopography.net/exhibition-2020/']
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
       
    }

    def parse(self, response):
        # yield {
        #     'title': response.css('title').extract(),
        #     'url': response.url,
        #     #'entries': response.css('div.portfolio-item').extract(),
        # }
        for entry in response.css('div.portfolio-item a.overlay-background::attr(href)'):
            yield response.follow(url=entry.get(), callback=self.parse_downloads)
    
    def parse_downloads(self, response):
        yield {
            'download': response.xpath("//a[text()=\'3840x2160\']/@href").extract()
        }


In [21]:

#Run the crawler
scrapydo.run_spider(wallpaperSpider, settings={
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

Downloading: https://desktopography.net/wp-content/uploads/bfi_thumb/Desk_2020_3-3bhqrglffx5kkwsqmewg74.jpg
saving to: Desk_2020_3-3bhqrglffx5kkwsqmewg74.jpg
Downloading: https://desktopography.net/wp-content/uploads/bfi_thumb/Desk_UndertheSea-3bhqtc705bwvr0drugd7nk.jpg
saving to: Desk_UndertheSea-3bhqtc705bwvr0drugd7nk.jpg
Downloading: https://desktopography.net/wp-content/uploads/bfi_thumb/wallpaper-4-Bloom-3bhqq37qqxokdug48z8rgg.jpg
saving to: wallpaper-4-Bloom-3bhqq37qqxokdug48z8rgg.jpg
Downloading: https://desktopography.net/wp-content/uploads/bfi_thumb/color_paradise-desktopography-2020-by-angelicafx-96DPI-3bhqrq57e5nvslizw8ya68.png
saving to: color_paradise-desktopography-2020-by-angelicafx-96DPI-3bhqrq57e5nvslizw8ya68.png
Downloading: https://desktopography.net/wp-content/uploads/bfi_thumb/ancientgreece-3bhqbk9gl7atnxrn5nrrpc.jpg
saving to: ancientgreece-3bhqbk9gl7atnxrn5nrrpc.jpg
Downloading: https://desktopography.net/wp-content/uploads/bfi_thumb/umer-tahir-anew-3bhqoenh5wrsu